# Download Webpages

In [30]:
import requests
import urllib.request
import bs4
from bs4 import BeautifulSoup

import uuid
import urllib.request
from tqdm import tqdm

# get user name like 'ec2-user' or 'root'
import getpass

# Generate URL of 104 Website

In [20]:
def get_url_job_list(num_pages=50):
    """
    Generate url of 104 website.
        Sample url: 
              ['https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=1&mode=s&jobsource=2018indexpoc',
               'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=2&mode=s&jobsource=2018indexpoc',
               'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=3&mode=s&jobsource=2018indexpoc',
               'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=10&mode=s&jobsource=2018indexpoc',
               'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=11&mode=s&jobsource=2018indexpoc']

    Return: return url lists
    """
    
    url_job_list = []
    
    for index in range(1, num_pages+1):
        url = 'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page={}&mode=s&jobsource=2018indexpoc'.format(index)
        url_job_list.append(url)
        
        
    return url_job_list


url_job_list = get_url_job_list()

In [53]:
url_job_list[17]

'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=18&mode=s&jobsource=2018indexpoc'

# Declare Parser Class

In [57]:
class ListParser():
    def __init__(self, url):
        
        # fetch html page and parse it.
        response = requests.get(url, timeout=10)
        self.content = BeautifulSoup(response.content, "html.parser")
        assert self.content is not None, 'Error: Empty content.'
        
        self.num_list = 0
        
        
    def get_lists(self):
        self.num_list = 0
        
        for element in self.content.findAll('article', attrs={'class': ['job-mode', 'js-job-item']}):
            if 'b-block--ad' in element.attrs['class']:
                continue
            else:
                self.num_list += 1
                yield element
        
        
class EntryParser():
    def __init__(self, element):
        """
        Args:
            element (bs4.element): element in BeautifulSoup
        """
        
        self.element = element
        
        
    def get_url(self):
        job = element.find('a', attrs={'class': 'js-job-link'})
        
        url_short = job['href']
        
        # fix invalid http link
        if 'javascript:' in url_short:
            return None
        
        
        return 'http:' + url_short
    
    
    def get_filename(self):
        
        # get a random unique id
        str_uuid = uuid.uuid4()
        
        company = element['data-cust-name'].replace('/', '-')
        title = element['data-job-name'].replace('/', '-')
        
        filename = '{}_{}_{}.html'.format(company, title, str_uuid)
        return filename
    

# Define Utility: Save Function

In [48]:
def save_page(url, filename):
    
    # fix problem when file name is Chinese.
    filename = filename.encode('utf8')
    
    urllib.request.urlretrieve(url, (filename))

# Main Program

In [65]:

# if the current server is aws, the folder is not in sub-folder
folder_name = '/newvolume/topic_modeling/dataset/' if getpass.getuser() == 'ec2-user' else './dataset/'
# folder_name = './dataset/'

# for url in url_job_list[:1]:
# for url in tqdm(url_job_list[:3]):
for url in tqdm(url_job_list):
    parser = ListParser(url)
    
    for element in parser.get_lists():
        entry = EntryParser(element)
        
        print(entry.get_url())
        print(folder_name + entry.get_filename())
        
        if entry.get_url() is None:
            continue
            
        save_page(entry.get_url(), folder_name + entry.get_filename())
        
#         if parser.num_list == 2:
#             break




  0%|          | 0/50 [00:00<?, ?it/s]

http://www.104.com.tw/job/6lng1?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/現觀科技股份有限公司_Machine Learning Engineer_bf8032f7-79ed-43c8-86f7-010477f1f587.html
http://www.104.com.tw/job/6arae?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/新馳科技有限公司_Machine Learning Technical Manager_1da4fa1c-afd4-4d25-8c34-60c92fa574b9.html
http://www.104.com.tw/job/6heww?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/奔騰網路科技有限公司_Machine Learning Engineer_2bdeea88-b55e-489b-8346-eadad45f21a7.html
http://www.104.com.tw/job/46izo?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/沛星互動科技股份有限公司_Machine Learning Scientist_80670350-15e5-47a0-a121-04280ec5c4ae.html
http://www.104.com.tw/job/6arac?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/新馳科技有限公司_Machine Learning Sr. Engineer_82f1e8e6-414b-463d-8e42-22c937cab27a.html
http://www.104.com.tw/job/6mst6?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/銓鴻資訊有限公司_Deep Learning 




  2%|▏         | 1/50 [00:08<06:45,  8.28s/it]

http://www.104.com.tw/job/6houm?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/工研院 _財團法人工業技術研究院_CITC_AI-Machine learning Engineer_5f42f1e7-1166-4bb2-926b-d38df1f82321.html
http://www.104.com.tw/job/5zi1u?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/英屬開曼群島商鼎峰智能股份有限公司台灣分公司_Machine Learning Data Scientist (Taipei)_ad70a8fe-7d4e-48f8-9660-a1cb6270a53b.html
http://www.104.com.tw/job/6c3kt?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/英屬開曼群島商鼎峰智能股份有限公司台灣分公司_Machine Learning Data Scientist (Hsinchu)_9715342e-d46d-440e-a554-a1113c90cee4.html
http://www.104.com.tw/job/61rxb?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/阿福股份有限公司_電腦視覺與機器學習研究員 Computer Vision-Machine Learning Researcher_41bda684-ca67-4222-b81a-c41e0d28893a.html
http://www.104.com.tw/job/6mqth?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/鴻華國際科技股份有限公司_Machine Learning - AI Application Engineer_28e36194-0946-452f-b891-ab07ab117ba9.html
http://www.104.com.tw/job/5ua2x?jobsource=2




  4%|▍         | 2/50 [00:15<06:24,  8.01s/it]

...




 88%|████████▊ | 44/50 [05:43<00:45,  7.51s/it]

http://www.104.com.tw/job/6ga4w?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/未來市股份有限公司_Hand motion and gesture Developer-Programmer_99348cf3-023c-4263-945d-f54158d0b1e3.html
http://www.104.com.tw/job/6hz4a?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/亞東工業氣體股份有限公司_(工業) 經營分析專員 Business  Analyst (台北內湖)_26f919e0-3d98-4f09-94ec-e4a1d53fa4de.html
http://www.104.com.tw/job/6jlii?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/Qualcomm Taiwan Corporation_台灣高通股份有限公司_Fingerprint Biometric Engineer (2019 應屆畢業生) (T1970568)_b709b0df-b11e-44c4-aada-4a387f77ae02.html
http://www.104.com.tw/job/3ugts?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/明基材料股份有限公司_【重點職缺】設備工程師(桃園總部)_caa9c688-a455-4150-a703-fb1faf35271c.html
http://www.104.com.tw/job/6k9u8?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/Advantech_研華股份有限公司_Gaming亞太市場開發(BDM-林口園區)_13a36aa8-b797-4f8c-8899-aa5f77e2226a.html
http://www.104.com.tw/job/5sm2z?jobsource=jol




 90%|█████████ | 45/50 [05:50<00:36,  7.37s/it]

http://www.104.com.tw/job/6jwmc?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/訊崴技術有限公司_【新竹_巨城百貨】品牌體驗顧問_457c93b1-e5a4-44fa-95eb-b1aab08a8f17.html
http://www.104.com.tw/job/6jwmb?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/訊崴技術有限公司_【苗栗_尚順】品牌體驗顧問_7f27764d-0a54-43e2-82de-0d6af6e4fea0.html
http://www.104.com.tw/job/6kyiy?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/訊崴技術有限公司_【高雄_新崛江商圈】品牌體驗顧問_83f78242-9ad2-4dfd-ad9f-90d26bd81308.html
http://www.104.com.tw/job/6kyj2?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/訊崴技術有限公司_【高雄_夢時代】品牌體驗顧問_6a25d8fc-168e-4a15-99e3-8403e6abd7c4.html
http://www.104.com.tw/job/6f5o7?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/台灣美光(台灣美光晶圓科技股份有限公司-台灣美光記憶體股份有限公司-美商美光亞太科技股份有限公司台灣分公司)_F11 Equipment Engineer設備工程師- Shift Equipment Engineer輪班設備工程師 - Taoyuan_c36fd33e-2862-4f53-95b7-6dd95fffa737.html


OSError: [Errno 36] File name too long: b'/newvolume/topic_modeling/dataset/\xe5\x8f\xb0\xe7\x81\xa3\xe7\xbe\x8e\xe5\x85\x89(\xe5\x8f\xb0\xe7\x81\xa3\xe7\xbe\x8e\xe5\x85\x89\xe6\x99\xb6\xe5\x9c\x93\xe7\xa7\x91\xe6\x8a\x80\xe8\x82\xa1\xe4\xbb\xbd\xe6\x9c\x89\xe9\x99\x90\xe5\x85\xac\xe5\x8f\xb8-\xe5\x8f\xb0\xe7\x81\xa3\xe7\xbe\x8e\xe5\x85\x89\xe8\xa8\x98\xe6\x86\xb6\xe9\xab\x94\xe8\x82\xa1\xe4\xbb\xbd\xe6\x9c\x89\xe9\x99\x90\xe5\x85\xac\xe5\x8f\xb8-\xe7\xbe\x8e\xe5\x95\x86\xe7\xbe\x8e\xe5\x85\x89\xe4\xba\x9e\xe5\xa4\xaa\xe7\xa7\x91\xe6\x8a\x80\xe8\x82\xa1\xe4\xbb\xbd\xe6\x9c\x89\xe9\x99\x90\xe5\x85\xac\xe5\x8f\xb8\xe5\x8f\xb0\xe7\x81\xa3\xe5\x88\x86\xe5\x85\xac\xe5\x8f\xb8)_F11 Equipment Engineer\xe8\xa8\xad\xe5\x82\x99\xe5\xb7\xa5\xe7\xa8\x8b\xe5\xb8\xab- Shift Equipment Engineer\xe8\xbc\xaa\xe7\x8f\xad\xe8\xa8\xad\xe5\x82\x99\xe5\xb7\xa5\xe7\xa8\x8b\xe5\xb8\xab - Taoyuan_3d37d16f-92b8-4a3e-b42c-54bcd47227d5.html'